# 3. Split data

In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import holidays
import geopandas as gpd
import libpysal as lps
import re
from shapely import wkb

In [2]:
ROOT = Path(os.path.abspath('')).resolve().parents[0]
DATA = os.path.join(ROOT, "data")
EXTERNAL_DATA = os.path.join(DATA, "external") 
INTERIM_DATA = os.path.join(DATA, "interim")
RAW_DATA = os.path.join(DATA, "raw")

In [4]:
df = gpd.read_parquet(
    os.path.join(INTERIM_DATA, 'df_geo_etl.geoparquet.gzip')
)
df

,created_at,start_date,start_hour,end_date,end_hour,lor,price,attempt,bicycle_type,group,...,age_14_15,age_15_18,age_18_21,age_25_27,age_55_60,age_60_64,age_80_85,poi_cnt,area_km2,poi_density_km2
0,2025-02-11,2025-10-31,15,2025-10-31,16,07601546,999,1,step_through,bicycle_theft,...,73.0,177.0,218.0,119.0,616.0,368.0,372.0,6.0,1.773447,3.383241
1,2025-02-11,2025-11-01,12,2025-11-01,18,01200522,1500,1,diamond_frame,bicycle_theft,...,76.0,234.0,294.0,385.0,772.0,554.0,305.0,128.0,0.639144,200.267843
2,2025-02-11,2025-11-01,0,2025-11-01,0,01300836,100,1,diamond_frame,cellar_attic_burglary,...,146.0,439.0,531.0,645.0,761.0,448.0,226.0,100.0,1.440714,69.410022
3,2025-02-11,2025-11-02,14,2025-11-02,16,03601243,240,1,mtb,bicycle_theft,...,55.0,127.0,114.0,213.0,462.0,207.0,119.0,71.0,0.329525,215.461750
4,2025-02-11,2025-10-22,12,2025-10-22,20,04501153,399,1,diamond_frame,bicycle_theft,...,56.0,231.0,185.0,245.0,510.0,288.0,217.0,65.0,0.585827,110.954225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35724,2024-02-01,2024-01-01,16,2024-01-01,20,04500938,1189,1,kids,bicycle_theft,...,53.0,168.0,177.0,244.0,598.0,315.0,359.0,23.0,0.517514,44.443283
35725,2024-02-01,2024-01-01,17,2024-01-02,12,04400727,2900,1,diamond_frame,bicycle_theft,...,59.0,173.0,192.0,221.0,593.0,389.0,430.0,19.0,0.914526,20.775792
35726,2024-01-01,2024-01-01,14,2024-01-01,16,06300632,899,1,multiple,bicycle_theft,...,173.0,484.0,468.0,249.0,1222.0,678.0,645.0,14.0,3.084440,4.538911
35727,2024-01-01,2024-01-01,19,2024-01-01,19,10300731,1,1,multiple,bicycle_theft,...,64.0,185.0,177.0,211.0,614.0,456.0,426.0,32.0,2.739947,11.679059


In [5]:
val_days  = 60
test_days = 60
gap_days  = 0
date_col  = "date" if "date" in df.columns else "start_date"

## Calculate time borders

In [6]:
d    = pd.to_datetime(df[date_col]).dt.normalize()
last = d.max()
gap  = pd.Timedelta(days=gap_days)

test_start = last - pd.Timedelta(days=test_days) + pd.Timedelta(days=1)
val_end    = test_start - pd.Timedelta(days=1) - gap
val_start  = val_end   - pd.Timedelta(days=val_days) + pd.Timedelta(days=1)
train_end  = val_start - pd.Timedelta(days=1) - gap

## Build masks

In [7]:
m_train = d <= train_end
m_val   = (d >= val_start) & (d <= val_end)
m_test  = d >= test_start

## Split dataset

In [8]:
df_train = df.loc[m_train].copy()
df_val   = df.loc[m_val].copy()
df_test  = df.loc[m_test].copy()

## Save split datasets

In [10]:
df_train.to_parquet(
    os.path.join(INTERIM_DATA, 'df_train.geoparquet.gzip'),
    compression='gzip'
)
df_val.to_parquet(
    os.path.join(INTERIM_DATA, 'df_val.geoparquet.gzip'),
    compression='gzip'
)
df_test.to_parquet(
    os.path.join(INTERIM_DATA, 'df_test.geoparquet.gzip'),
    compression='gzip'
)